# Day 17: Conway Cubes

As your flight slowly drifts through the sky, the Elves at the Mythical Information Bureau at the North Pole contact you. They'd like some help debugging a malfunctioning experimental energy source aboard one of their super-secret imaging satellites.

The experimental energy source is based on cutting-edge technology: a set of Conway Cubes contained in a pocket dimension! When you hear it's having problems, you can't help but agree to take a look.

The pocket dimension contains an infinite 3-dimensional grid. At every integer 3-dimensional coordinate (x,y,z), there exists a single cube which is either active or inactive.

In the initial state of the pocket dimension, almost all cubes start inactive. The only exception to this is a small flat region of cubes (your puzzle input); the cubes in this region start in the specified active (#) or inactive (.) state.

The energy source then proceeds to boot up by executing six cycles.

Each cube only ever considers its neighbors: any of the 26 other cubes where any of their coordinates differ by at most 1. For example, given the cube at x=1,y=2,z=3, its neighbors include the cube at x=2,y=2,z=2, the cube at x=0,y=2,z=3, and so on.

During a cycle, all cubes simultaneously change their state according to the following rules:

If a cube is active and exactly 2 or 3 of its neighbors are also active, the cube remains active. Otherwise, the cube becomes inactive.
If a cube is inactive but exactly 3 of its neighbors are active, the cube becomes active. Otherwise, the cube remains inactive.
The engineers responsible for this experimental energy source would like you to simulate the pocket dimension and determine what the configuration of cubes should be at the end of the six-cycle boot process.

For example, consider the following initial state:

```
.#.
..#
###
```
Even though the pocket dimension is 3-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a 3x3x1 region of the 3-dimensional space.)

Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given z coordinate (and the frame of view follows the active cells in each cycle):

Before any cycles:
```
z=0
.#.
..#
###
```

After 1 cycle:
```
z=-1
#..
..#
.#.

z=0
#.#
.##
.#.

z=1
#..
..#
.#.
```

After 2 cycles:
```
z=-2
.....
.....
..#..
.....
.....

z=-1
..#..
.#..#
....#
.#...
.....

z=0
##...
##...
#....
....#
.###.

z=1
..#..
.#..#
....#
.#...
.....

z=2
.....
.....
..#..
.....
.....
```

After 3 cycles:
```
z=-2
.......
.......
..##...
..###..
.......
.......
.......

z=-1
..#....
...#...
#......
.....##
.#...#.
..#.#..
...#...

z=0
...#...
.......
#......
.......
.....##
.##.#..
...#...

z=1
..#....
...#...
#......
.....##
.#...#.
..#.#..
...#...

z=2
.......
.......
..##...
..###..
.......
.......
.......
```
After the full six-cycle boot process completes, 112 cubes are left in the active state.

Starting with your given initial configuration, simulate six cycles. How many cubes are left in the active state after the sixth cycle?

In [40]:
input = """######.#
#.###.#.
###.....
#.####..
##.#.###
.######.
###.####
######.#"""

In [41]:
import numpy as np

ACTIVE = '#'
INACTIVE = '.'

In [42]:
def set_coordinate(grid, x, y, z, v):
    if x not in grid:
        grid[x] = {}
    if y not in grid[x]:
        grid[x][y] = {}
    grid[x][y][z] = v
    
def get_coordinate(grid, x, y, z):
    try:
        return grid[x][y][z]
    except KeyError:
        return INACTIVE
    
def active_neighbours(grid, x, y, z):
    num_active = 0
    for xn in (x-1, x, x+1):
        for yn in (y-1, y, y+1):
            for zn in (z-1, z, z+1):
                if x == xn and y == yn and z == zn:
                    continue
                if get_coordinate(grid, xn, yn, zn) == ACTIVE:
                    num_active += 1
                    
    return num_active
        
    
grid = {}
for y, row in enumerate(input.splitlines()):
    for x, char in enumerate(row):
        set_coordinate(grid, x, y, 0, char)
        
from copy import deepcopy
def make_step(grid):
    new_grid = deepcopy(grid)
    x_values = grid.keys()
    y_values = [c for r in grid.values() for c in r.keys()]
    z_values = [l for r in grid.values() for c in r.values() for l in c.keys()]
    min_x, max_x = min(x_values), max(x_values)
    min_y, max_y = min(y_values), max(y_values)
    min_z, max_z = min(z_values), max(z_values)
    for x in range(min_x - 1, max_x + 2):
        for y in range(min_y - 1, max_y + 2):
            for z in range(min_z - 1, max_z + 2):
                neighbours_active = active_neighbours(grid, x, y, z)
                is_active = get_coordinate(grid, x, y, z) == ACTIVE
                if is_active and neighbours_active not in (2, 3):
                    set_coordinate(new_grid, x, y, z, INACTIVE)
                elif not is_active and neighbours_active == 3:
                    set_coordinate(new_grid, x, y, z, ACTIVE)
    return new_grid

for step in range(6):
    grid = make_step(grid)


num_active = 0
for x in grid:
    for y in grid[x]:
        for z in grid[x][y]:
            num_active += get_coordinate(grid, x, y, z) == ACTIVE
            
print(num_active)

448


# Part Two

For some reason, your simulated results don't match what the experimental energy source engineers expected. Apparently, the pocket dimension actually has four spatial dimensions, not three.

The pocket dimension contains an infinite 4-dimensional grid. At every integer 4-dimensional coordinate (x,y,z,w), there exists a single cube (really, a hypercube) which is still either active or inactive.

Each cube only ever considers its neighbors: any of the 80 other cubes where any of their coordinates differ by at most 1. For example, given the cube at x=1,y=2,z=3,w=4, its neighbors include the cube at x=2,y=2,z=3,w=3, the cube at x=0,y=2,z=3,w=4, and so on.

The initial state of the pocket dimension still consists of a small flat region of cubes. Furthermore, the same rules for cycle updating still apply: during each cycle, consider the number of active neighbors of each cube.

For example, consider the same initial state as in the example above. Even though the pocket dimension is 4-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a 3x3x1x1 region of the 4-dimensional space.)

Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given z and w coordinate:

Before any cycles:
```
z=0, w=0
.#.
..#
###
```

After 1 cycle:
```
z=-1, w=-1
#..
..#
.#.

z=0, w=-1
#..
..#
.#.

z=1, w=-1
#..
..#
.#.

z=-1, w=0
#..
..#
.#.

z=0, w=0
#.#
.##
.#.

z=1, w=0
#..
..#
.#.

z=-1, w=1
#..
..#
.#.

z=0, w=1
#..
..#
.#.

z=1, w=1
#..
..#
.#.
```

After 2 cycles:
```
z=-2, w=-2
.....
.....
..#..
.....
.....

z=-1, w=-2
.....
.....
.....
.....
.....

z=0, w=-2
###..
##.##
#...#
.#..#
.###.

z=1, w=-2
.....
.....
.....
.....
.....

z=2, w=-2
.....
.....
..#..
.....
.....

z=-2, w=-1
.....
.....
.....
.....
.....

z=-1, w=-1
.....
.....
.....
.....
.....

z=0, w=-1
.....
.....
.....
.....
.....

z=1, w=-1
.....
.....
.....
.....
.....

z=2, w=-1
.....
.....
.....
.....
.....

z=-2, w=0
###..
##.##
#...#
.#..#
.###.

z=-1, w=0
.....
.....
.....
.....
.....

z=0, w=0
.....
.....
.....
.....
.....

z=1, w=0
.....
.....
.....
.....
.....

z=2, w=0
###..
##.##
#...#
.#..#
.###.

z=-2, w=1
.....
.....
.....
.....
.....

z=-1, w=1
.....
.....
.....
.....
.....

z=0, w=1
.....
.....
.....
.....
.....

z=1, w=1
.....
.....
.....
.....
.....

z=2, w=1
.....
.....
.....
.....
.....

z=-2, w=2
.....
.....
..#..
.....
.....

z=-1, w=2
.....
.....
.....
.....
.....

z=0, w=2
###..
##.##
#...#
.#..#
.###.

z=1, w=2
.....
.....
.....
.....
.....

z=2, w=2
.....
.....
..#..
.....
.....
```
After the full six-cycle boot process completes, 848 cubes are left in the active state.

Starting with your given initial configuration, simulate six cycles in a 4-dimensional space. How many cubes are left in the active state after the sixth cycle?



In [43]:
def set_coordinate(grid, x, y, z, w, v):
    if x not in grid:
        grid[x] = {}
    if y not in grid[x]:
        grid[x][y] = {}
    if z not in grid[x][y]:
        grid[x][y][z] = {}
    grid[x][y][z][w] = v
    
def get_coordinate(grid, x, y, z, w):
    try:
        return grid[x][y][z][w]
    except KeyError:
        return INACTIVE
    
def active_neighbours(grid, x, y, z, w):
    num_active = 0
    for xn in (x-1, x, x+1):
        for yn in (y-1, y, y+1):
            for zn in (z-1, z, z+1):
                for wn in (w-1, w, w+1):
                    if x == xn and y == yn and z == zn and w == wn:
                        continue
                    if get_coordinate(grid, xn, yn, zn, wn) == ACTIVE:
                        num_active += 1
                    
    return num_active
        
    
grid = {}
for y, row in enumerate(input.splitlines()):
    for x, char in enumerate(row):
        set_coordinate(grid, x, y, 0, 0, char)
        
from copy import deepcopy
def make_step(grid):
    new_grid = deepcopy(grid)
    x_values = grid.keys()
    y_values = [c for r in grid.values() for c in r.keys()]
    z_values = [l for r in grid.values() for c in r.values() for l in c.keys()]
    w_values = [hl for r in grid.values() for c in r.values() for l in c.values() for hl in l.keys()]
    min_x, max_x = min(x_values), max(x_values)
    min_y, max_y = min(y_values), max(y_values)
    min_z, max_z = min(z_values), max(z_values)
    min_w, max_w = min(w_values), max(w_values)
    for x in range(min_x - 1, max_x + 2):
        for y in range(min_y - 1, max_y + 2):
            for z in range(min_z - 1, max_z + 2):
                for w in range(min_w - 1, max_w + 2):
                    neighbours_active = active_neighbours(grid, x, y, z, w)
                    is_active = get_coordinate(grid, x, y, z, w) == ACTIVE
                    if is_active and neighbours_active not in (2, 3):
                        set_coordinate(new_grid, x, y, z, w, INACTIVE)
                    elif not is_active and neighbours_active == 3:
                        set_coordinate(new_grid, x, y, z, w, ACTIVE)
    return new_grid

for step in range(6):
    grid = make_step(grid)


num_active = 0
for x in grid:
    for y in grid[x]:
        for z in grid[x][y]:
            for w in grid[x][y][z]:
                num_active += get_coordinate(grid, x, y, z, w) == ACTIVE
            
print(num_active)

2400
